In [1]:
from collections import namedtuple
import os
os.chdir("../")
os.getcwd()

'p:\\DEEPCNNClassifier'

In [2]:
from src.deepClassifier.constants import *
from src.deepClassifier.utils import read_yaml, create_directories
from configs import *

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [4]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        params = self.params
        prepare_base_model = self.config.prepare_base_model
        training_data = os.path.join(self.config.unzip_dir, 'PetImages')
        create_directories([
            Path(training.root_dir)
                            ])
        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path= Path(prepare_base_model.updated_base_model_path),
            training_data= Path(training_data),
            params_epochs= params.EPOCHS,
            params_batch_size= params.BATCH_SIZE,
            params_is_augmentation= params.AUGMENTATION,
            params_image_size= params.IMAGE_SIZE,
        )

        return training_config

In [5]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

class Training:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path
        )
    def train_valid_generator(self):
        data_generator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20,
        )
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )

        valid_datagenerator = tf.keras.preprocessing.Datagenerator(
            **data_generator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_dictionary(
            directory = self.config.training_data,
            subset = 'validation',
            shuffle = False,
            **dataflow_kwargs
        )
        if self.config.params_is_augmentation:
            training_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                shear_range = 0.2,
                zoom_range = 0.2,
                horizontal_flip = True,
                **data_generator_kwargs
            )
        else:
            training_datagenerator  = valid_datagenerator

        self.train_generator = training_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = 'training',
            shuffle = True,
            **dataflow_kwargs
        )
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

        
    def train(self, callbacks: list):
        self.steps_per_epoch = self.train_generator // self.train_generator.batch_size
        self.validation_steps = self.valid_generator // self.valid_generator.batch_size
        self.model.fit(
            self.train_generator,
            steps_per_epoch = self.steps_per_epoch,
            epochs = self.config.params_epochs,
            validation_data = self.valid_generator,
            validation_steps= self.validation_steps,
            callbacks = callbacks



        )

NameError: name 'PrepareCallbacksConfig' is not defined

In [ ]:

try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
    config = ConfigurationManager()
    training_config = config.get_prepare_callback_config()
    training= Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callbacks=callback_list
    )
except Exception as e:
    raise e